In [3]:
import pandas as pd
import mysql.connector
from typing import Dict, List
from dotenv import load_dotenv # pip install python-dotenv
import os

In [4]:
# Cargar variables desde el archivo .env
load_dotenv(dotenv_path=".env")

True

In [5]:
def connect_to_database():
    """Establece la conexión con la base de datos MySQL"""
    host = os.getenv("host")
    user = os.getenv("user")
    password = os.getenv("password")
    database = os.getenv("database")

    return mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )

In [9]:

def process_excel(file_path: str) -> tuple[Dict[str, tuple], List[tuple]]:
    """
    Procesa el archivo CSV y retorna los datos procesados para países y equipos
    """
    # Leer el CSV
    df = pd.read_excel(file_path)
    
    # Procesar países únicos
    countries_dict = {}
    country_id = 1
    
    for _, row in df.iterrows():
        if row['countryCode'] not in countries_dict:
            countries_dict[row['countryCode']] = (
                country_id,
                row['countryName'],
                row['countryCode'],
                None  # countryImage
            )
            country_id += 1
    
    # Procesar equipos
    teams_data = []
    for _, row in df.iterrows():
        country_id = countries_dict[row['countryCode']][0]
        teams_data.append((
            int(row['teamId']),
            row['teamName'],
            None,  # teamShortName
            row['imageUrl'],
            country_id
        ))
    
    return countries_dict, teams_data

def insert_countries(cursor, countries_data: Dict[str, tuple]):
    """
    Inserta o actualiza los datos de países en la base de datos.
    Si el país ya existe (mismo countryId), actualiza sus datos.
    """
    upsert_query = """
    INSERT INTO countries (countryId, countryName, countryShortName, countryImage)
    VALUES (%s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        countryName = VALUES(countryName),
        countryShortName = VALUES(countryShortName),
        countryImage = VALUES(countryImage)
    """
    countries_values = [country_data for country_data in countries_data.values()]
    
    try:
        cursor.executemany(upsert_query, countries_values)
        print(f"Procesados {len(countries_values)} países")
        print(f"Insertados/Actualizados: {cursor.rowcount // 2} registros")
    except Exception as e:
        print(f"Error al procesar países: {str(e)}")
        raise

def insert_teams(cursor, teams_data: List[tuple]):
    """
    Inserta o actualiza los datos de equipos en la base de datos.
    Si el equipo ya existe (mismo teamId), actualiza sus datos.
    """
    upsert_query = """
    INSERT INTO teams (teamId, teamName, teamShortName, teamImage, teamCountryId)
    VALUES (%s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        teamName = VALUES(teamName),
        teamShortName = COALESCE(VALUES(teamShortName), teamShortName),
        teamImage = VALUES(teamImage),
        teamCountryId = VALUES(teamCountryId)
    """
    
    try:
        cursor.executemany(upsert_query, teams_data)
        print(f"Procesados {len(teams_data)} equipos")
        print(f"Insertados/Actualizados: {cursor.rowcount // 2} registros")
    except Exception as e:
        print(f"Error al procesar equipos: {str(e)}")
        raise

# Función auxiliar para verificar los cambios realizados
def verify_changes(cursor, table_name: str):
    """
    Verifica y muestra los registros insertados/actualizados en una tabla
    """
    query = f"SELECT * FROM {table_name}"
    cursor.execute(query)
    records = cursor.fetchall()
    print(f"\nRegistros en {table_name}: {len(records)}")
    for record in records[:5]:  # Muestra los primeros 5 registros como ejemplo
        print(record)


In [7]:
def main():
    try:
        connection = connect_to_database()
        cursor = connection.cursor()
        
        # Procesar el CSV
        countries_data, teams_data = process_excel('teams.xlsx')
        
        # Insertar/Actualizar datos
        insert_countries(cursor, countries_data)
        insert_teams(cursor, teams_data)
        
        # Verificar cambios
        verify_changes(cursor, 'countries')
        verify_changes(cursor, 'teams')
        
        # Confirmar cambios
        connection.commit()
        print("\nTodos los cambios fueron confirmados exitosamente")
        
    except Exception as error:
        print(f"Error: {error}")
        connection.rollback()
        print("Se realizó rollback de los cambios")
        
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("Conexión cerrada")

In [10]:
if __name__ == "__main__":
    main()

Procesados 16 países
Insertados/Actualizados: 8 registros
Procesados 36 equipos
Insertados/Actualizados: 18 registros

Registros en countries: 16
(1, 'Austria', None, 'at')
(2, 'Germany', None, 'de')
(3, 'Netherlands', None, 'nl')
(4, 'England', None, 'gb-eng')
(5, 'Slovakia', None, 'sk')

Registros en teams: 36
(13, 'Arsenal', None, 'https://d2zywfiolv4f83.cloudfront.net/img/teams/13.png', 4)
(24, 'Aston Villa', None, 'https://d2zywfiolv4f83.cloudfront.net/img/teams/24.png', 4)
(26, 'Liverpool', None, 'https://d2zywfiolv4f83.cloudfront.net/img/teams/26.png', 4)
(36, 'Bayer Leverkusen', None, 'https://d2zywfiolv4f83.cloudfront.net/img/teams/36.png', 2)
(37, 'Bayern Munich', None, 'https://d2zywfiolv4f83.cloudfront.net/img/teams/37.png', 2)

Todos los cambios fueron confirmados exitosamente
Conexión cerrada


CARGAR A MANO EN LA TABLA DE TEAMS LOS SHORT NAME
CARGAR EL PAIS